In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
import praw
import json
import requests
import pprint

In [2]:
client = MongoClient()

db = client['reddit']
comments = db['comments']

myclient = MongoClient()
mydb = myclient['myreddit']
mycomments = mydb['mycomments']

In [15]:
comments_month = comments.count()
print('Total comments from month: {}'.format(comments_month))

comments.find_one()

Total comments from month: 12316144


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


{'_id': ObjectId('5e505081129d2eb1218c93ac'),
 'link_id': 't3_j4zx3',
 'score_hidden': False,
 'score': 1,
 'archived': True,
 'author_flair_text': None,
 'subreddit': 'fffffffuuuuuuuuuuuu',
 'body': '"$2, would you take that deal? I\'d take that deal"',
 'author': 'DorkyDude',
 'distinguished': None,
 'parent_id': 't3_j4zx3',
 'id': 'c298mtc',
 'subreddit_id': 't5_2qqlo',
 'controversiality': 0,
 'gilded': 0,
 'downs': 0,
 'retrieved_on': 1427415708,
 'name': 't1_c298mtc',
 'ups': 1,
 'edited': False,
 'author_flair_css_class': None,
 'created_utc': '1312156800'}

In [5]:
# TODO
# Grab post text based on comment
# Looks like text posts and link posts are different.
# Looks like top-level comments and child comments are different.

pol1 = comments.find({'subreddit': 'politics'}).limit(3)
# print(pol1.next())
for i in pol1:
    pprint.pprint(i)

{'_id': ObjectId('5e505081129d2eb1218c93ae'),
 'archived': True,
 'author': 'chaon93',
 'author_flair_css_class': None,
 'author_flair_text': None,
 'body': 'a baton is more likely to kill someone than a taser',
 'controversiality': 0,
 'created_utc': '1312156800',
 'distinguished': None,
 'downs': 0,
 'edited': False,
 'gilded': 0,
 'id': 'c298mti',
 'link_id': 't3_j4sdr',
 'name': 't1_c298mti',
 'parent_id': 't1_c295rb2',
 'retrieved_on': 1427415708,
 'score': 1,
 'score_hidden': False,
 'subreddit': 'politics',
 'subreddit_id': 't5_2cneq',
 'ups': 1}
{'_id': ObjectId('5e505081129d2eb1218c93fb'),
 'archived': True,
 'author': '[deleted]',
 'author_flair_css_class': None,
 'author_flair_text': None,
 'body': '[deleted]',
 'controversiality': 0,
 'created_utc': '1312156818',
 'distinguished': None,
 'downs': 0,
 'edited': False,
 'gilded': 0,
 'id': 'c298mvo',
 'link_id': 't3_j4zxt',
 'name': 't1_c298mvo',
 'parent_id': 't3_j4zxt',
 'retrieved_on': 1428197302,
 'score': -3,
 'score_hid

In [4]:
politics_count = comments.find({'subreddit': 'politics'}).count()
print('Count of /r/politics comments: {}'.format(politics_count))
considered_count = comments.find({'subreddit': {'$in': ['politics', 'sports', 'worldnews', 'The_Donald']}}).count()
print('Count of comments in considered subs: {}'.format(considered_count))
print('Fraction of all comments this month: {:1.2%}'.format(considered_count/comments_month))

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


Count of /r/politics comments: 307820


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Count of comments in considered subs: 479505
Fraction of all comments this month: 3.89%


In [5]:
with open('../keys/reddit_appid.txt') as f:
    APP_ID = f.read().rstrip()
with open('../keys/reddit_secret.txt') as f:
    APP_SECRET = f.read().rstrip()
with open('../keys/reddit.txt') as f:
    pw = f.read().rstrip()

In [33]:
# base_url = 'https://www.reddit.com/'
# data = {'grant_type': 'password', 'username': 'some_username', 'password': pw}
# auth = requests.auth.HTTPBasicAuth(APP_ID, APP_SECRET)
# r = requests.post(base_url + 'api/v1/access_token',
#                   data=data,
#                   headers={'user-agent': 'APP-NAME by REDDIT-USERNAME'},
#                   auth=auth)
# d = r.json()
# print(d)

In [23]:
# token = 'bearer ' + d['access_token']

# base_url = 'https://oauth.reddit.com'

# headers = {'Authorization': token, 'User-Agent': 'APP-NAME by REDDIT-USERNAME', 'id': 'dm6zumf'}
# response = requests.get(base_url + '/api/v1/me', headers=headers)

# if response.status_code == 200:
#     print(response.json()['name'], response.json()['comment_karma'])

Make empty reddit comments table, reddit posts table

For each month:

    For each top-level comment: 
        pull relevant info into a reddit comments table

    For each unique post_id in reddit comments table not in the reddit posts table:
        Group ~20 ids, get post data from PRAW, save to reddit posts table

In [22]:
cursor = comments.find({'subreddit': 
                            {'$in': ['politics', 'sports', 'worldnews', 'The_Donald']}, 
                        '$expr': 
                            {'$eq': ['$link_id', '$parent_id']}}, 
                       {'_id': 1, 'author': 1, 'body': 1, 'created_utc': 1, 'id': 1, 'link_id': 1, 
                            'parent_id': 1, 'score': 1, 'subreddit': 1})

cursor.count()
# for c in cursor:
#     pprint.pprint(c)
# #     mycomments.insert_one(c)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


128764

In [46]:
reddit = praw.Reddit(client_id=APP_ID,
                     client_secret=APP_SECRET,
                     user_agent='script:my.project:v1.0.0 (by /u/Someone')

ids = ['aoe4pk']
ids2 = [i if i.startswith('t3_') else f't3_{i}' for i in ids]

for submission in reddit.info(ids2):
    d = {'title': submission.title, 'datetime': submission.created_utc, 'score': submission.score, 'is_self': submission.is_self,
         'sub': submission.subreddit.display_name, 'permalink': submission.permalink}
    print(d)

{'title': '[PRAW] Getting multiple submissions using by_id', 'datetime': 1549610404.0, 'score': 2, 'is_self': True, 'sub': 'redditdev', 'permalink': '/r/redditdev/comments/aoe4pk/praw_getting_multiple_submissions_using_by_id/'}


In [46]:
response = requests.get(base_url + '/api/info.json?id=t3_6wddne', headers=headers)
pprint.pprint(response.json())

{'data': {'after': None,
          'before': None,
          'children': [{'data': {'all_awardings': [],
                                 'allow_live_comments': False,
                                 'approved_at_utc': None,
                                 'approved_by': None,
                                 'archived': True,
                                 'author': 'taylorkline',
                                 'author_flair_background_color': '',
                                 'author_flair_css_class': None,
                                 'author_flair_richtext': [],
                                 'author_flair_template_id': None,
                                 'author_flair_text': 'Bot Developer',
                                 'author_flair_text_color': 'dark',
                                 'author_flair_type': 'text',
                                 'author_fullname': 't2_13jrwt',
                                 'author_patreon_flair': False,
                 

In [47]:
response = requests.get(base_url + '/api/info.json?id=t3_j4zx3', headers=headers)
pprint.pprint(response.json())

{'data': {'after': None,
          'before': None,
          'children': [{'data': {'all_awardings': [],
                                 'allow_live_comments': False,
                                 'approved_at_utc': None,
                                 'approved_by': None,
                                 'archived': True,
                                 'author': '[deleted]',
                                 'author_flair_background_color': '',
                                 'author_flair_css_class': None,
                                 'author_flair_template_id': None,
                                 'author_flair_text': None,
                                 'author_flair_text_color': 'dark',
                                 'awarders': [],
                                 'banned_at_utc': None,
                                 'banned_by': None,
                                 'can_gild': False,
                                 'can_mod_post': False,
                  

In [11]:
client.close()